In [1]:
dat="""col1\tcol2\tcol3\tdate1
1\t2.3\tabc\t2021-11-17 12:33:12
10\t4.3\tdef\t2021-11-18 14:56:42
20\t5.3\tsgh123\t2021-11-19 17:23:52"""
file_name = raw".\filename.csv"
open(file_name,"w") do f 
    println(f, dat) 
end

In [2]:
;more $file_name

col1    col2    col3    date1
1       2.3     abc     2021-11-17 12:33:12
10      4.3     def     2021-11-18 14:56:42
20      5.3     sgh123  2021-11-19 17:23:52


In [3]:
using CSV, DataFrames, Dates
df = CSV.read(file_name, DataFrame;
              dateformat="y-m-d H:M:S", types = Dict(:date1 => DateTime))

Row,col1,col2,col3,date1
,Int64,Float64,String7,DateTime
1,1,2.3,abc,2021-11-17T12:33:12
2,10,4.3,def,2021-11-18T14:56:42
3,20,5.3,sgh123,2021-11-19T17:23:52


In [4]:
df.date1[1]

2021-11-17T12:33:12

In [5]:
df.year = year.(df.date1)
df.day = day.(df.date1)
df

Row,col1,col2,col3,date1,year,day
,Int64,Float64,String7,DateTime,Int64,Int64
1,1,2.3,abc,2021-11-17T12:33:12,2021,17
2,10,4.3,def,2021-11-18T14:56:42,2021,18
3,20,5.3,sgh123,2021-11-19T17:23:52,2021,19


In [6]:
a = rand(3,3)

3×3 Matrix{Float64}:
 0.450185  0.00937441  0.910652
 0.719387  0.693428    0.385493
 0.799998  0.891112    0.465691

In [7]:
df[:, Not(:col3)]

Row,col1,col2,date1,year,day
,Int64,Float64,DateTime,Int64,Int64
1,1,2.3,2021-11-17T12:33:12,2021,17
2,10,4.3,2021-11-18T14:56:42,2021,18
3,20,5.3,2021-11-19T17:23:52,2021,19


In [8]:
using XLSX
println("Current folder : ",pwd())

Current folder : C:\AAABIBLIOTEKA\CAMK\2024_Julia_EC_EconomicDepShort\2_Working_with_Data_Sources


In [9]:
XLSX.openxlsx(joinpath(pwd(),"my_new_file.xlsx"), mode="w") do xf
    sheet = xf[1]
    XLSX.rename!(sheet, "Spreadsheet name")
    sheet["A1"] = "this"
    sheet["A2"] = "is a"
    sheet["A3"] = "new file"
    sheet["A4"] = 100

    sheet["A7"] = collect(1:5)   #slide to right
    sheet["A8", dim=2] = collect(1:4) # slide to right

    
    sheet["C1", dim=1] = collect(1:4) # slide downwards

    sheet["E3:G5"] = collect(reshape(1:9,(3,3)))
    
end

3×3 Matrix{Int64}:
 1  4  7
 2  5  8
 3  6  9

In [10]:
import XLSX

xf = XLSX.readxlsx(joinpath(pwd(),"my_new_file.xlsx"))

XLSXFile("C:\AAABIBLIOTEKA\CAMK\2024_Julia_EC_EconomicDepShort\2_Working_with_Data_Sources\my_new_file.xlsx") containing 1 Worksheet
            sheetname size          range        
-------------------------------------------------
     Spreadsheet name 8x7           A1:G8        


In [11]:
XLSX.sheetnames(xf)

1-element Vector{String}:
 "Spreadsheet name"

In [12]:
sh = xf[XLSX.sheetnames(xf)[1]] # get a reference to a Worksheet

8×7 XLSX.Worksheet: ["Spreadsheet name"](A1:G8) 

In [13]:
dim = XLSX.get_dimension(sh)
start = (dim.start.row_number, dim.start.column_number)
end_ = (dim.stop.row_number, dim.stop.column_number)
start, end_

((1, 1), (8, 7))

In [14]:
for row in start[1]:end_[1]
    for col in start[2]:end_[2]
        print(sh[row,col])
        print("\t")
    end
    println()
end        

this	missing	1	missing	missing	missing	missing	
is a	missing	2	missing	missing	missing	missing	
new file	missing	3	missing	1	4	7	
100	missing	4	missing	2	5	8	
missing	missing	missing	missing	3	6	9	
missing	missing	missing	missing	missing	missing	missing	
1	2	3	4	5	missing	missing	
1	2	3	4	missing	missing	missing	


In [15]:
sh[:]

8×7 Matrix{Any}:
    "this"       missing  1          missing   missing   missing   missing
    "is a"       missing  2          missing   missing   missing   missing
    "new file"   missing  3          missing  1         4         7
 100             missing  4          missing  2         5         8
    missing      missing   missing   missing  3         6         9
    missing      missing   missing   missing   missing   missing   missing
   1            2         3         4         5          missing   missing
   1            2         3         4          missing   missing   missing

In [16]:
m = Matrix{Union{Int,Float64,String,Missing}}(undef,(end_[1],end_[2]))
m .= sh[:]

8×7 Matrix{Union{Missing, Float64, Int64, String}}:
    "this"       missing  1          missing   missing   missing   missing
    "is a"       missing  2          missing   missing   missing   missing
    "new file"   missing  3          missing  1         4         7
 100             missing  4          missing  2         5         8
    missing      missing   missing   missing  3         6         9
    missing      missing   missing   missing   missing   missing   missing
   1            2         3         4         5          missing   missing
   1            2         3         4          missing   missing   missing

In [17]:
xf[XLSX.sheetnames(xf)[1]*"!A2:B4"]

3×2 Matrix{Any}:
    "is a"      missing
    "new file"  missing
 100            missing

In [18]:
using DataFrames
df = DataFrame(a=1:5, b=string.("Hello",1:5), c=6.5:10.5, d=[11,22,33,missing,missing] )

Row,a,b,c,d
,Int64,String,Float64,Int64?
1,1,Hello1,6.5,11
2,2,Hello2,7.5,22
3,3,Hello3,8.5,33
4,4,Hello4,9.5,missing
5,5,Hello5,10.5,missing


In [19]:
typeof(df.d)

Vector{Union{Missing, Int64}} (alias for Array{Union{Missing, Int64}, 1})

In [20]:
XLSX.writetable(joinpath(pwd(),"df.xlsx"), overwrite=true, 
    WORKSHEET_A=(collect(DataFrames.eachcol(df)), DataFrames.names(df)),
    MYWS_B=(collect(DataFrames.eachcol(df)), DataFrames.names(df)),
)
# after comma new worksheets can be added tj. WORKSHEET_B=(collect(DataFrames.eachcol(df)), DataFrames.names(df)) itd.

In [21]:
data2 = XLSX.readtable(joinpath(pwd(),"df.xlsx"),"WORKSHEET_A";infer_eltypes=true)

XLSX.DataTable(Any[[1, 2, 3, 4, 5], ["Hello1", "Hello2", "Hello3", "Hello4", "Hello5"], [6.5, 7.5, 8.5, 9.5, 10.5], Union{Missing, Int64}[11, 22, 33, missing, missing]], [:a, :b, :c, :d], Dict(:a => 1, :b => 2, :d => 4, :c => 3))

In [22]:
DataFrame(data2)

Row,a,b,c,d
,Int64,String,Float64,Int64?
1,1,Hello1,6.5,11
2,2,Hello2,7.5,22
3,3,Hello3,8.5,33
4,4,Hello4,9.5,missing
5,5,Hello5,10.5,missing


In [23]:
using HTTP
response = HTTP.get("https://szufel.pl/pliki/plik.txt");

In [24]:
@show response.status


response.status = 200


200

In [25]:
response.body

408-element Vector{UInt8}:
 0x7b
 0x0a
 0x20
 0x20
 0x22
 0x66
 0x69
 0x72
 0x73
 0x74
 0x4e
 0x61
 0x6d
    ⋮
 0x65
 0x22
 0x3a
 0x20
 0x6e
 0x75
 0x6c
 0x6c
 0x0a
 0x7d
 0x0a
 0x0a

In [26]:
data = String(response.body)
println(data)

{
  "firstName": "John",
  "lastName": "Smith",
  "isAlive": true,
  "age": 25,
  "address": {
    "streetAddress": "21 2nd Street",
    "city": "New York",
    "state": "NY",
    "postalCode": "10021-3100"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "212 555-1234"
    },
    {
      "type": "office",
      "number": "646 555-4567"
    }
  ],
  "children": [],
  "spouse": null
}




In [27]:
using JSON3
dat = JSON3.read(data)

JSON3.Object{Base.CodeUnits{UInt8, String}, Vector{UInt64}} with 8 entries:
  :firstName    => "John"
  :lastName     => "Smith"
  :isAlive      => true
  :age          => 25
  :address      => {…
  :phoneNumbers => Object[{…
  :children     => Union{}[]
  :spouse       => nothing

In [28]:
#dat = Dict(JSON3.read(data))
dat[:phoneNumbers][1]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 2 entries:
  :type   => "home"
  :number => "212 555-1234"

In [29]:
dat[:phoneNumbers][1]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 2 entries:
  :type   => "home"
  :number => "212 555-1234"

In [30]:
dat[:phoneNumbers][1][:number]

"212 555-1234"

In [31]:
file_name = tempname()*".json"
open(file_name,"w") do f
    JSON3.write(f, dat)
end
@show file_name;

file_name = "C:\\Users\\pszuf\\AppData\\Local\\Temp\\jl_ETyhopSDcJ.json"


In [6]:
using Arrow, DataFrames, Random
df = DataFrame(a=1:5, b=rand(5), c='a':'e', d=[randstring(4) for _ in 1:5])

Row,a,b,c,d
,Int64,Float64,Char,String
1,1,0.175403,a,ylJq
2,2,0.282239,b,cRXL
3,3,0.808952,c,juEM
4,4,0.983031,d,eH4B
5,5,0.190268,e,YxD5


In [7]:
Arrow.write("mytbl.arrow", df)

"mytbl.arrow"

In [10]:
table = Arrow.Table("mytbl.arrow")

Arrow.Table with 5 rows, 4 columns, and schema:
 :a  Int64
 :b  Float64
 :c  Char
 :d  String

In [11]:
df2 = DataFrame(table)

Row,a,b,c,d
,Int64,Float64,Char,String
1,1,0.175403,a,ylJq
2,2,0.282239,b,cRXL
3,3,0.808952,c,juEM
4,4,0.983031,d,eH4B
5,5,0.190268,e,YxD5


In [13]:
try 
    df2[1, :a] = 11
catch ex
    println(ex)
end

ReadOnlyMemoryError()
